1. 엑셀 파일 처리: 엑셀 파일에서 데이터를 추출하고, JSON 파일로 저장합니다.
2. JSON 파일 검증: 변환된 JSON 파일에서 데이터를 샘플링하고, 필드 유효성을 확인합니다.
3. 데이터 전처리: 형태소 분석과 불용어 제거를 통해 데이터를 정제합니다.
4. 토큰화: 정제된 데이터를 BERT Tokenizer로 토큰화합니다.
5. 데이터셋 준비: 데이터셋을 정의하고, DataLoader로 데이터를 배치 단위로 나눕니다.
6. 데이터 확인: 배치된 데이터를 확인하고, 학습에 사용할 준비가 되었는지 검증합니다.

In [1]:
import os
import sys
import urllib.request
import pandas as pd
import requests

import warnings
# 경고 메시지 무시 설정
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [2]:
# !pip install ipywidgets

In [3]:
base_dir = r"/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/01.원천데이터"
save_dir = r"/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터"

1. 엑셀 파일 읽기 및 JSON 변환

In [4]:
import pandas as pd
import json
import glob
import os

# 엑셀 파일들이 들어 있는 최상위 폴더 경로 (리눅스 경로로 수정)
base_dir = r"/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/01.원천데이터"

# 모든 하위 폴더 내 엑셀 파일 경로 검색
excel_files = glob.glob(os.path.join(base_dir, '**', '*.xlsx'), recursive=True)

qa_data = []  # JSON으로 저장할 데이터 리스트

# 모든 엑셀 파일 읽기
for file_path in excel_files:
    try:
        # 엑셀 파일 읽기
        df = pd.read_excel(file_path)

        question_info = None  # 현재 질문 정보를 저장할 변수

        # 각 행을 순회하며 질문(Q)과 답변(A) 추출
        for idx, row in df.iterrows():
            if row['질의응답'] == 'Q':  # 질문일 때
                question_info = {
                    "question": row['발언내용'],
                    "회의번호": row['회의번호'],
                    "질의응답번호": row['질의응답번호'],
                    "회의구분": row['회의구분'],
                    "위원회": row['위원회'],
                    "회의일자": row['회의일자'],
                    "질문자": row['의원ID'],
                    "질문자_ISNI": row['ISNI']
                }
            elif row['질의응답'] == 'A' and question_info is not None:  # 답변일 때
                answer_info = {
                    "answer": row['발언내용'],
                    "답변자": row['의원ID'],
                    "답변자_ISNI": row['ISNI']
                }
                
                # 유효성 검사: 질문과 답변이 있는지 확인
                if not question_info['question'] or not answer_info['answer']:
                    print(f"파일 {file_path}의 {idx}번째 항목에서 질문 또는 답변이 누락되었습니다. 건너뜁니다.")
                    continue

                # 질문과 답변을 연결하여 하나의 JSON 객체로 만듦
                qa_data.append({**question_info, **answer_info})
                question_info = None  # 사용 후 질문 정보를 초기화

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except pd.errors.EmptyDataError:
        print(f"빈 파일이므로 건너뜁니다: {file_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생 ({file_path}): {e}")

# 라벨링 데이터 파일 저장 경로 (리눅스 경로로 수정)
save_dir = r"/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터"
save_path = os.path.join(save_dir, '라벨링데이터.json')

# 디렉토리가 존재하지 않으면 생성
os.makedirs(save_dir, exist_ok=True)

# JSON 파일로 저장
try:
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(qa_data, f, ensure_ascii=False, indent=4)
    print("모든 파일 처리가 완료되었습니다.")
except Exception as e:
    print(f"JSON 파일 저장 중 오류 발생: {e}")


KeyboardInterrupt: 

2. 변환된 JSON 파일 검증

In [ ]:
import json
import os

# 라벨링된 JSON 파일 경로 (리눅스/WSL 경로로 수정)
file_path = "/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/라벨링데이터.json"

# JSON 파일 로드 및 예외 처리
try:
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_path}")

    # JSON 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

        # 1. JSON 파일에서 일부 샘플을 확인 (처음 5개 데이터 출력)
        print("데이터 샘플 확인 (처음 5개 질문-답변 쌍):")
        for i, item in enumerate(data[:5]):
            print(f"샘플 {i + 1}:")
            print(json.dumps(item, indent=4, ensure_ascii=False))
            print("-" * 50)

        # 2. 총 질문-답변 쌍의 개수 확인
        print(f"\n총 질문-답변 쌍의 개수: {len(data)}")

        # 3. 필드 유효성 검사
        required_fields = {"question", "answer", "회의번호", "질의응답번호", "회의구분", "위원회", "회의일자", "질문자", "질문자_ISNI", "답변자", "답변자_ISNI"}
        print("\n필드 유효성 검사:")
        for i, item in enumerate(data):
            missing_fields = required_fields - item.keys()  # 필요한 필드 중 누락된 필드 찾기
            if missing_fields:
                print(f"항목 {i}에 필요한 필드가 누락되었습니다: {missing_fields}")
        
        # 4. 질문-답변 쌍의 일관성 확인
        print("\n질문-답변 쌍 일관성 확인:")
        for i, item in enumerate(data):
            if not item.get("question") or not item.get("answer"):
                print(f"항목 {i}에 질문 또는 답변이 없습니다.")

except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_path}")
except json.JSONDecodeError:
    print("JSON 파일을 읽는 중 오류가 발생했습니다. 파일 형식이 올바른지 확인해주세요.")
except Exception as e:
    print(f"오류 발생: {e}")


3. 데이터 전처리: 형태소 분석과 불용어 제거

In [ ]:
import re
from konlpy.tag import Mecab

# Mecab 형태소 분석기 로드
mecab = Mecab()

# 불용어 리스트 정의
stop_words = [
    "것", "있다", "하다", "입니다", "그리고", "하지만", "또한", "그런데", "저는", "우리는", "그래서", "이것", "저것", "그것",
    "다시", "모든", "각각", "모두", "어느", "몇몇", "이런", "저런", "그런", "어떤", "특히", "즉", "또", "이후", "때문에", "통해서",
    "같은", "많은", "따라서", "등", "경우", "관련", "대해", "의해", "이기", "대한", "그리고", "라고", "이라는", "에서", "부터", "까지",
    "와", "과", "으로", "에", "의", "를", "가", "도", "로", "에게", "만", "뿐", "듯", "제", "내", "저", "그", "할", "수", "있", "같",
    "되", "보다", "아니", "아닌", "이", "있어서", "입니다", "있습니다", "합니다", "입니까", "같습니다", "아닙니다", "라는", "그러므로", 
    "입니다만", "때문입니다", "라고요", "그러하다", "하고", "이와"
]

# 긴 텍스트를 슬라이싱하는 함수 정의
def slice_long_text(text, max_length=512, overlap=50):
    """
    긴 텍스트를 슬라이싱하여 나눔
    :param text: 원본 텍스트
    :param max_length: 최대 토큰 길이
    :param overlap: 슬라이싱할 때 겹치는 토큰 수
    :return: 슬라이싱된 텍스트 리스트
    """
    tokens = mecab.morphs(text)  # 형태소 분석을 통한 토큰화
    
    sliced_texts = []
    start_idx = 0

    # 토큰의 길이가 최대 길이를 초과하면 슬라이싱
    while start_idx < len(tokens):
        end_idx = min(start_idx + max_length, len(tokens))
        sliced_texts.append(tokens[start_idx:end_idx])
        start_idx = end_idx - overlap  # overlap 만큼 겹치게 슬라이싱

    # 슬라이싱된 텍스트 리스트 반환
    return [" ".join(slice) for slice in sliced_texts]

# 데이터 전처리 함수 정의
def preprocess_text_with_slicing(text):
    """
    긴 텍스트를 슬라이싱하여 각 슬라이스에 대해 전처리 수행
    :param text: 원본 텍스트
    :return: 전처리된 텍스트
    """
    # 1. 긴 텍스트를 슬라이싱
    sliced_texts = slice_long_text(text)

    # 2. 슬라이싱된 각 텍스트를 전처리
    preprocessed_slices = []
    for slice in sliced_texts:
        # 특수 문자 및 공백 제거
        slice = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", slice)
        slice = re.sub(r"\s+", " ", slice).strip()

        # 형태소 분석 및 불용어 제거
        tokens = mecab.morphs(slice)
        filtered_tokens = [word for word in tokens if word not in stop_words]
        
        # 정제된 단어들로 다시 결합하여 저장
        preprocessed_slices.append(" ".join(filtered_tokens))

    # 3. 모든 슬라이스를 합쳐서 반환
    return " ".join(preprocessed_slices)

In [ ]:

# 전처리 확인을 위한 샘플 데이터
sample_data = [
    "저는 오늘 국회 회의에 참석하여 중요한 발표를 했습니다.",
    "정부 정책에 대한 국민들의 반응이 매우 긍정적입니다.",
    "이 법안이 통과될 경우, 앞으로의 경제 상황이 나아질 것입니다."
]

# 원본 데이터와 전처리된 데이터 비교
print("전처리 전과 후 비교:\n")
for i, sentence in enumerate(sample_data):
    preprocessed = preprocess_text_with_slicing(sentence)
    print(f"원본: {sentence}")
    print(f"전처리 후: {preprocessed}")
    print("-" * 50)


4. BERT Tokenizer를 사용한 토큰화

In [11]:
from transformers import BertTokenizer

# BERT Tokenizer 로드 (한국어 지원되는 BERT 모델)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 긴 텍스트를 슬라이싱하는 함수 정의
def slice_long_text(text, max_length=512, overlap=50):
    """
    긴 텍스트를 슬라이싱하여 나눔
    :param text: 원본 텍스트
    :param max_length: 최대 토큰 길이
    :param overlap: 슬라이싱할 때 겹치는 토큰 수
    :return: 슬라이싱된 텍스트 리스트
    """
    tokens = mecab.morphs(text)  # 형태소 분석을 통한 토큰화
    
    sliced_texts = []
    start_idx = 0

    # 토큰의 길이가 최대 길이를 초과하면 슬라이싱
    while start_idx < len(tokens):
        end_idx = min(start_idx + max_length, len(tokens))
        sliced_texts.append(tokens[start_idx:end_idx])
        start_idx = end_idx - overlap  # overlap 만큼 겹치게 슬라이싱

    # 슬라이싱된 텍스트 리스트 반환
    return [" ".join(slice) for slice in sliced_texts]

# 토큰화 함수 정의 (긴 텍스트 슬라이싱을 포함)
def tokenize_text(question, answer):
    """
    질문과 답변을 토큰화하며 긴 텍스트 처리
    :param question: 질문 텍스트
    :param answer: 답변 텍스트
    :return: 토큰화된 input_ids와 attention_mask
    """
    # 1. 질문과 답변을 각각 슬라이싱
    sliced_questions = slice_long_text(question)
    sliced_answers = slice_long_text(answer)

    input_ids_list = []
    attention_mask_list = []

    # 2. 슬라이싱된 각 질문과 답변에 대해 토큰화
    for q_slice, a_slice in zip(sliced_questions, sliced_answers):
        inputs = tokenizer(
            q_slice,
            a_slice,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'  # PyTorch 텐서 형식으로 반환
        )

        # 각 슬라이스의 토큰 ID 및 패딩 마스크 추가
        input_ids_list.append(inputs['input_ids'].squeeze())  # 문장의 토큰 ID
        attention_mask_list.append(inputs['attention_mask'].squeeze())  # 문장의 패딩 여부 (0 또는 1)

    # 3. 슬라이스된 결과를 리스트로 반환
    return input_ids_list, attention_mask_list


5. 데이터셋 준비: 데이터셋을 정의하고, DataLoader로 데이터를 배치 단위로 나눕니다.

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스 정의
class QADataset(Dataset):
    def __init__(self, data):
        self.data = data  # 데이터는 전처리된 JSON 데이터로 가정

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # 질문과 답변 전처리
        question = preprocess_text(item['question'])
        answer = preprocess_text(item['answer'])

        # 질문과 답변 토큰화 (긴 텍스트 슬라이싱 포함)
        input_ids_list, attention_mask_list = tokenize_text(question, answer)

        # 여러 슬라이스가 있을 경우, 리스트로 반환
        return {
            'input_ids': input_ids_list,
            'attention_mask': attention_mask_list
        }

# 데이터셋 생성
dataset = QADataset(data)

# DataLoader로 데이터를 배치 단위로 준비
# 여기서는 collate_fn을 사용하여 슬라이스된 텍스트들을 처리할 수 있게 설정
def collate_fn(batch):
    input_ids_batch = []
    attention_mask_batch = []

    # 배치의 각 샘플을 순회
    for sample in batch:
        input_ids_batch.extend(sample['input_ids'])  # 각 슬라이스의 input_ids 추가
        attention_mask_batch.extend(sample['attention_mask'])  # 각 슬라이스의 attention_mask 추가

    # 텐서 형태로 변환 (이 경우 padding 등의 처리가 필요할 수 있음)
    input_ids_batch = torch.stack(input_ids_batch)
    attention_mask_batch = torch.stack(attention_mask_batch)

    return {
        'input_ids': input_ids_batch,
        'attention_mask': attention_mask_batch
    }

# DataLoader 정의
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)


In [ ]:
6. 데이터 확인: 배치된 데이터를 확인하고, 학습에 사용할 준비가 되었는지 검증합니다.

In [13]:
# 데이터셋에서 샘플 확인
for batch in dataloader:
    print("Input IDs:", batch['input_ids'])
    print("Attention Mask:", batch['attention_mask'])
    break  # 첫 번째 배치만 확인

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Input IDs: tensor([[   101,   9655, 118673,  ...,      0,      0,      0],
        [   101,   9341,  17730,  ...,      0,      0,      0],
        [   101,   9706,  33305,  ...,   9482,  48345,    102],
        ...,
        [   101,  93222,   9356,  ...,      0,      0,      0],
        [   101,   8924,  30873,  ...,   9557,   9043,    102],
        [   101,   9765,  11287,  ...,      0,      0,      0]])
Attention Mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])


1. Input IDs와 Attention Mask 해석
* Input IDs:
    * 101: [CLS] 토큰입니다. BERT 모델에서는 입력의 시작을 의미합니다.
    * 0: 패딩 토큰입니다. 문장이 BERT 모델의 최대 길이(여기서는 512)보다 짧으면 그 남은 부분을 0으로 채워줍니다. 이는 모델이 실제 단어가 없는 부분을 무시하도록 하는 역할을 합니다.
    * 그 외 숫자들은 BERT 토크나이저에 의해 변환된 단어들의 토큰 ID입니다.
* Attention Mask:
    * 1: 실제 단어가 있는 부분입니다. BERT 모델이 이 부분을 처리합니다.
    * 0: 패딩 부분입니다. 모델은 이 부분을 무시합니다.